Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
import pickle
import joblib

Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Data Preparation and Feature Extraction:



In [3]:
players_21=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_21.csv')

In [4]:
players_21.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,18944.000000,18944.000000,18944.000000,1.870700e+04,18719.000000,18944.000000,18944.000000,18944.000000,18719.000000,18719.000000,...,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,2083.000000
mean,226232.328917,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,181.190773,75.016892,47908.905551,1.355468,...,57.978674,46.470017,47.581767,45.546505,16.446052,16.236486,16.103357,16.225982,16.551309,37.203553
std,27166.574284,7.002278,6.109985,7.743775e+06,19893.149956,4.697354,6.825672,7.057140,53906.733776,0.739015,...,12.118390,20.169591,21.402461,20.953997,17.577332,16.845480,16.519399,17.017341,17.878121,10.714523
min,41.000000,47.000000,47.000000,9.000000e+03,500.000000,16.000000,155.000000,50.000000,1.000000,1.000000,...,12.000000,3.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,210028.750000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,461.000000,1.000000,...,50.000000,29.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000,28.000000
50%,232301.500000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1913.000000,1.000000,...,59.000000,52.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,37.000000
75%,246745.750000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,110986.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,258970.000000,93.000000,95.000000,1.855000e+08,560000.000000,53.000000,206.000000,110.000000,114899.000000,4.000000,...,96.000000,94.000000,93.000000,90.000000,90.000000,92.000000,93.000000,91.000000,90.000000,65.000000


In [5]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


In [6]:
players_21.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [7]:
print(players_21.shape[1])

110


In [8]:
#droping columns with more than 30% null values
threshold = 0.3
total_rows = len(players_21)
for column in players_21.columns:
  miss_percentage = players_21[column].isnull().sum()/total_rows

  if miss_percentage > threshold:
    players_21 = players_21.drop(column, axis=1)

In [9]:
print(players_21.shape[1])

102


In [10]:
#droping useless columns
useless_columns = ['sofifa_id', 'player_url', 'long_name', 'dob', 'short_name',
                     'body_type', 'real_face',
                   'player_face_url', 'club_logo_url',  'nation_flag_url','club_flag_url']

In [11]:
new_players_21 = players_21.drop(useless_columns, axis = 1)

In [12]:
#separating the data into numeric and textual forms:
numeric_data = pd.DataFrame()
textual_data = pd.DataFrame()

for column in new_players_21:
  if new_players_21[column].dtype in ['int64', 'float64']:
    numeric_data[column] = new_players_21[column]
  else:
    textual_data[column] = new_players_21[column]

In [13]:
numeric_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  int64  
 1   potential                    18944 non-null  int64  
 2   value_eur                    18707 non-null  float64
 3   wage_eur                     18719 non-null  float64
 4   age                          18944 non-null  int64  
 5   height_cm                    18944 non-null  int64  
 6   weight_kg                    18944 non-null  int64  
 7   club_team_id                 18719 non-null  float64
 8   league_level                 18719 non-null  float64
 9   club_jersey_number           18719 non-null  float64
 10  club_contract_valid_until    18719 non-null  float64
 11  nationality_id               18944 non-null  int64  
 12  weak_foot                    18944 non-null  int64  
 13  skill_moves     

In [14]:
textual_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   player_positions  18944 non-null  object
 1   club_name         18719 non-null  object
 2   league_name       18719 non-null  object
 3   club_position     18719 non-null  object
 4   club_joined       17961 non-null  object
 5   nationality_name  18944 non-null  object
 6   preferred_foot    18944 non-null  object
 7   work_rate         18944 non-null  object
 8   ls                18944 non-null  object
 9   st                18944 non-null  object
 10  rs                18944 non-null  object
 11  lw                18944 non-null  object
 12  lf                18944 non-null  object
 13  cf                18944 non-null  object
 14  rf                18944 non-null  object
 15  rw                18944 non-null  object
 16  lam               18944 non-null  object
 17  cam         

In [15]:
#imputing the numerical data
columns_with_missing_values = numeric_data.columns[numeric_data.isnull().any().tolist()]

for column in columns_with_missing_values:
   numeric_data[column].fillna(numeric_data[column].mean(), inplace=True)


In [16]:
#imputing the categorical data
textual_columns_with_missing_values = textual_data.columns[textual_data.isnull().any().tolist()]

for column in textual_columns_with_missing_values:
  textual_data[column].fillna(textual_data[column].mode()[0], inplace=True)

In [17]:
numeric_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  int64  
 1   potential                    18944 non-null  int64  
 2   value_eur                    18944 non-null  float64
 3   wage_eur                     18944 non-null  float64
 4   age                          18944 non-null  int64  
 5   height_cm                    18944 non-null  int64  
 6   weight_kg                    18944 non-null  int64  
 7   club_team_id                 18944 non-null  float64
 8   league_level                 18944 non-null  float64
 9   club_jersey_number           18944 non-null  float64
 10  club_contract_valid_until    18944 non-null  float64
 11  nationality_id               18944 non-null  int64  
 12  weak_foot                    18944 non-null  int64  
 13  skill_moves     

In [18]:
#encoding the textual values
label_encoder = LabelEncoder()

for column in textual_data:
  textual_data[column] = label_encoder.fit_transform(textual_data[column])

In [19]:
textual_data

,player_positions,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,ls,st,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,539,238,41,0,8,6,0,7,215,215,...,136,136,136,133,110,59,59,59,110,18
1,587,361,24,13,1135,123,1,1,216,216,...,110,110,110,126,104,66,66,66,104,20
2,560,240,19,27,289,122,1,2,214,214,...,136,136,136,119,104,96,96,96,104,18
3,331,461,17,14,888,20,1,2,206,206,...,116,116,116,139,110,52,52,52,110,20
4,10,407,15,19,487,14,1,0,203,203,...,232,232,232,204,185,158,158,158,185,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,55,138,6,28,1769,30,1,4,20,20,...,39,39,39,39,44,46,46,46,44,9
18940,149,560,6,21,1769,30,1,4,35,35,...,52,52,52,48,49,44,44,44,49,9
18941,149,608,6,21,1769,30,1,4,40,40,...,49,49,49,46,48,43,43,43,48,1
18942,560,444,14,28,1769,48,1,8,43,43,...,21,21,21,30,26,10,10,10,26,7


In [20]:
cleaned_players_21 = pd.concat([numeric_data, textual_data], axis=1)
cleaned_players_21

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,93,93,103500000.0,560000.0,33,170,72,241.0,1.0,10.0,...,136,136,136,133,110,59,59,59,110,18
1,92,92,63000000.0,220000.0,35,187,83,45.0,1.0,7.0,...,110,110,110,126,104,66,66,66,104,20
2,91,91,111000000.0,240000.0,31,184,80,21.0,1.0,9.0,...,136,136,136,119,104,96,96,96,104,18
3,91,91,132000000.0,270000.0,28,175,68,73.0,1.0,10.0,...,116,116,116,139,110,52,52,52,110,20
4,91,91,129000000.0,370000.0,29,181,70,10.0,1.0,17.0,...,232,232,232,204,185,158,158,158,185,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47,52,70000.0,1000.0,21,177,70,112165.0,1.0,25.0,...,39,39,39,39,44,46,46,46,44,9
18940,47,53,70000.0,1000.0,21,174,68,112540.0,1.0,37.0,...,52,52,52,48,49,44,44,44,49,9
18941,47,47,45000.0,2000.0,28,185,79,111774.0,1.0,33.0,...,49,49,49,46,48,43,43,43,48,1
18942,47,67,130000.0,500.0,17,171,58,1920.0,4.0,27.0,...,21,21,21,30,26,10,10,10,26,7


FEATURE ENGINEERING PROCESS:

In [21]:
#feature engineering with randomforestclassifier - feature importance

#choosing X and y
X = cleaned_players_21.drop('overall',axis=1)
y = cleaned_players_21['overall']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rforrest_classifier = RandomForestClassifier(n_estimators=112, max_depth=12, criterion='entropy')

In [22]:
rforrest_classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=112)

In [23]:
feature_importance = rforrest_classifier.feature_importances_

In [24]:
for name, score in zip(X.columns, feature_importance):
  print(name,'=', score)

potential = 0.036783797238638595
value_eur = 0.09522571724333477
wage_eur = 0.022358371785261125
age = 0.02803152138150469
height_cm = 0.0024225740733186257
weight_kg = 0.002585102706056897
club_team_id = 0.0029112013967244926
league_level = 0.0004782668745621455
club_jersey_number = 0.0025698299144318796
club_contract_valid_until = 0.0012404932372374077
nationality_id = 0.0025378872184246247
weak_foot = 0.0006609743568581656
skill_moves = 0.0007933784995314809
international_reputation = 0.001155508049142962
release_clause_eur = 0.06822632722098747
pace = 0.0027430816297148034
shooting = 0.004386683478816325
passing = 0.002580551454331657
dribbling = 0.00487892762263976
defending = 0.01271881797621373
physic = 0.0063263070890560635
attacking_crossing = 0.004819361085799183
attacking_finishing = 0.004062065598083098
attacking_heading_accuracy = 0.005653092113414746
attacking_short_passing = 0.003021386000712734
attacking_volleys = 0.0030076148233491593
skill_dribbling = 0.00307200343938

In [25]:
#sorting the feature importance
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df.iloc[0:20]

,Feature,Importance
1,value_eur,0.095226
14,release_clause_eur,0.068226
0,potential,0.036784
34,movement_reactions,0.031059
87,rcb,0.030622
85,lcb,0.030313
86,cb,0.030245
3,age,0.028032
78,rm,0.027328
74,lm,0.027045


In [26]:
#using correlation matrix
cmatrix = pd.DataFrame(cleaned_players_21.corr())
cmatrix['overall'].iloc[0:21].sort_values(ascending = False)

overall                      1.000000
passing                      0.662090
potential                    0.636366
release_clause_eur           0.614344
dribbling                    0.596558
wage_eur                     0.589060
value_eur                    0.555192
age                          0.468197
shooting                     0.454391
international_reputation     0.440987
skill_moves                  0.381369
defending                    0.333616
weak_foot                    0.222814
pace                         0.188862
weight_kg                    0.146242
club_contract_valid_until    0.122961
height_cm                    0.031579
nationality_id              -0.098959
club_jersey_number          -0.187180
club_team_id                -0.207319
league_level                -0.215148
Name: overall, dtype: float64

In [27]:
#picking from the top 10 features according to the feature importance and the correlation matrix:
selected_features = ['potential', 'value_eur','age','release_clause_eur','dribbling','shooting','wage_eur'
]

In [28]:
##training and testing next:
#scaling
scaler = StandardScaler()
X = cleaned_players_21[selected_features]
X_scaled = scaler.fit_transform(X)
y = pd.DataFrame(cleaned_players_21['overall'])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

training:

In [30]:
#Random Forrest Regressor with grid search
model = RandomForestRegressor(random_state=42)

In [31]:
model.fit(X_train, y_train)
initial_predictions = model.predict(X_test)
initial_rmse = np.sqrt(mean_squared_error(y_test, initial_predictions))
initial_rmse

<ipython-input-31-b108a8a5d22d>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


0.48239464366916096

In [32]:
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [33]:
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=4)

In [34]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWa

GridSearchCV(cv=4, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [5, 10, 15],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 150, 200]},
             scoring='neg_mean_squared_error')

In [35]:
best_parameters = grid_search.best_params_
best_parameters

{'max_depth': 15,
 'max_features': 'auto',
 'min_samples_split': 2,
 'n_estimators': 150}

In [36]:
best_model = grid_search.best_estimator_
best_model

RandomForestRegressor(max_depth=15, max_features='auto', n_estimators=150,
                      random_state=42)

In [37]:
#evaluating the model with cross_val_score
score = cross_val_score(best_model,X,y,scoring='neg_mean_squared_error', cv=5)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWa

In [38]:
rmse_scores = np.sqrt(-score)
rmse_scores

array([5.39894998, 1.71216285, 1.26568315, 1.47637044, 5.22255846])

In [39]:
final_rfr_model = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_split=2, random_state=42)

In [40]:
final_rfr_model.fit(X_train,y_train)

<ipython-input-40-60758a4abb0a>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  final_rfr_model.fit(X_train,y_train)


RandomForestRegressor(max_depth=15, n_estimators=200, random_state=42)

In [41]:
final_pred=final_rfr_model.predict(X_test)

In [42]:
rmse = np.sqrt(mean_squared_error(y_test,final_pred))
print("Mean RMSE:", rmse)

Mean RMSE: 0.4853097687103814


In [43]:
final_rfr_model.score(X_test, y_test)

0.995003167479579

In [44]:
#xgboost model:
xgb_model = xgb.XGBRegressor()

In [45]:
xgb_model.fit(X_train, y_train)
xgb_initial_predictions = xgb_model.predict(X_test)
xgb_initial_rmse = np.sqrt(mean_squared_error(y_test, xgb_initial_predictions))
xgb_initial_rmse

0.47895190872082805

***Hyperparameter Tuning***


In [46]:
param_grid1 = {
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 500],
    'max_depth': [3, 5],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}


In [47]:
#root mean score error scorer
scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
cv = KFold(n_splits=3, shuffle=True, random_state=42)
xgb_grid_search = GridSearchCV(xgb_model, param_grid=param_grid1, scoring=scorer,cv=cv)

In [48]:
xgb_grid_search.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    inte...
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.1, 0.01], 'max_depth': [3, 5],
                         'min_child_weight': [1, 3], 'n_estimators': [100, 500],
                         'subsample': [0.8, 1.0]},
             scoring=make_scorer(<lambda>, greater_is_better=False))

In [51]:
xgb_best_parameters = xgb_grid_search.best_params_
xgb_best_parameters

{'colsample_bytree': 1.0,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 3,
 'n_estimators': 500,
 'subsample': 0.8}

In [52]:
#best xgb model
xgb_best_model = xgb_grid_search.best_estimator_
xgb_best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [53]:
xgb_cv_scores = cross_val_score(xgb_best_model, X, y, cv=cv, scoring=scorer)
xgb_rmse_scores = -xgb_cv_scores

In [54]:
print("RMSE Scores for Each Fold:", xgb_rmse_scores)

RMSE Scores for Each Fold: [0.46409202 0.45756641 0.46460639]


In [55]:
#final xgb model
final_xgb_model = xgb_best_model

In [56]:
#training the final model
final_xgb_model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [57]:
#xgb_prediction
xgb_prediction = final_xgb_model.predict(X_test)
xgb_prediction

array([59.964664, 63.62095 , 58.012024, ..., 71.73561 , 62.91342 ,
       61.37455 ], dtype=float32)

In [58]:
mean_absolute_error(xgb_prediction,y_test)

0.283956115432703

In [59]:
xgb_mean_rmse = np.mean(xgb_rmse_scores)
print("Mean RMSE:", xgb_mean_rmse)

Mean RMSE: 0.4620882726309247


In [60]:
final_xgb_model.score(X_test, y_test)

0.9953917578198526

In [61]:
#gradient boost
gb_model = GradientBoostingRegressor()

In [62]:
gb_model.fit(X_train, y_train)
gb_initial_predictions = gb_model.predict(X_test)
gb_initial_rmse = np.sqrt(mean_squared_error(y_test, gb_initial_predictions))
gb_initial_rmse

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7675640865331985

**Hyperparameter tuning:**

In [63]:
parameter_grid = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
}

gb_scorer = scorer #from the xgboost regressor

cv = cv # from xgboost cv

gb_grid_search = GridSearchCV(gb_model, parameter_grid, scoring=gb_scorer, cv=cv)

In [64]:
gb_grid_search.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 150, 200]},
             scoring=make_scorer(<lambda>, greater_is_better=False))

In [65]:
gb_best_parameters = gb_grid_search.best_params_
gb_best_parameters

{'learning_rate': 0.1,
 'max_depth': 7,
 'min_samples_split': 5,
 'n_estimators': 200}

In [66]:
#best model
gb_best_model = gb_grid_search.best_estimator_

In [67]:
#final GradientBoostingRegressor Model
final_gb_model = gb_best_model

In [68]:
gb_prediction = final_gb_model.predict(X_test)
gb_prediction

array([59.93185   , 63.66926498, 58.05531748, ..., 72.21378455,
       62.93900424, 61.42781624])

In [69]:
gb_cv_scores = cross_val_score(final_gb_model, X, y, cv=cv, scoring=scorer)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [70]:
gb_rmse_score = -gb_cv_scores
gb_rmse_score

array([0.452536  , 0.43580976, 0.45182917])

In [71]:
#mean root mean square error for GradientboostingRegression
mean_rmse = np.mean(gb_rmse_score)
print("Mean RMSE:", mean_rmse)

Mean RMSE: 0.44672497763525026


In [72]:
final_gb_model.score(X_test, y_test)

0.995618887172753

**TESTING WITH UNSEEN DATA (PLAYERS 22)** WITH trained MODEL: ***GradientBoostingRegressor Model, XGBOOST, RandomForrestRegressor***

Cleaning the players 22

In [73]:
players_22=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')

<ipython-input-73-d535ce371154>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')


In [74]:
print(players_22.shape[1])

110


In [75]:
#droping columns with more than 30% null values
threshold = 0.3
total_rows = len(players_22)
for column in players_22.columns:
  miss_percentage = players_22[column].isnull().sum()/total_rows

  if miss_percentage > threshold:
    players_22 = players_22.drop(column, axis=1)

In [76]:
print(players_22.shape[1])

102


In [77]:
#dropping useless columns
useless_columns22 = ['sofifa_id', 'player_url', 'long_name', 'dob', 'short_name',
                     'body_type', 'real_face',
                   'player_face_url', 'club_logo_url',  'nation_flag_url','club_flag_url']

In [78]:
new_players_22 = players_22.drop(useless_columns22, axis = 1)

In [79]:
new_players_22.head()

,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,73.0,Paris Saint-Germain,...,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3
1,ST,92,92,119500000.0,270000.0,32,185,81,21.0,FC Bayern München,...,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
2,"ST, LW",91,91,45000000.0,270000.0,36,187,83,11.0,Manchester United,...,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3
3,"LW, CAM",91,91,129000000.0,270000.0,29,175,68,73.0,Paris Saint-Germain,...,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3
4,"CM, CAM",91,91,125500000.0,350000.0,30,181,70,10.0,Manchester City,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3


In [80]:
#separating the data into numeric and textual forms:
numeric_data22 = pd.DataFrame()
textual_data22 = pd.DataFrame()

for column in new_players_22:
  if new_players_22[column].dtype in ['int64', 'float64']:
    numeric_data22[column] = new_players_22[column]
  else:
    textual_data22[column] = new_players_22[column]

In [81]:
numeric_data22.isnull().sum()

overall                           0
potential                         0
value_eur                        74
wage_eur                         61
age                               0
height_cm                         0
weight_kg                         0
club_team_id                     61
league_level                     61
club_jersey_number               61
club_contract_valid_until        61
nationality_id                    0
weak_foot                         0
skill_moves                       0
international_reputation          0
release_clause_eur             1176
pace                           2132
shooting                       2132
passing                        2132
dribbling                      2132
defending                      2132
physic                         2132
attacking_crossing                0
attacking_finishing               0
attacking_heading_accuracy        0
attacking_short_passing           0
attacking_volleys                 0
skill_dribbling             

In [82]:
textual_data22.isnull().sum()

player_positions       0
club_name             61
league_name           61
club_position         61
club_joined         1163
nationality_name       0
preferred_foot         0
work_rate              0
ls                     0
st                     0
rs                     0
lw                     0
lf                     0
cf                     0
rf                     0
rw                     0
lam                    0
cam                    0
ram                    0
lm                     0
lcm                    0
cm                     0
rcm                    0
rm                     0
lwb                    0
ldm                    0
cdm                    0
rdm                    0
rwb                    0
lb                     0
lcb                    0
cb                     0
rcb                    0
rb                     0
gk                     0
dtype: int64

In [83]:
#imputing the numerical data
columns_with_missing_values22 = numeric_data22.columns[numeric_data22.isnull().any().tolist()]

for column in columns_with_missing_values22:
   numeric_data22[column].fillna(numeric_data22[column].mean(), inplace=True)


In [84]:
#imputing the categorical data
textual_columns_with_missing_values22 = textual_data22.columns[textual_data22.isnull().any().tolist()]

for column in textual_columns_with_missing_values22:
  textual_data22[column].fillna(textual_data22[column].mode()[0], inplace=True)

In [85]:
#encoding the textual values
label_encoder22 = LabelEncoder()

for column in textual_data22:
  textual_data22[column] = label_encoder22.fit_transform(textual_data22[column])

In [86]:
cleaned_players_22 = pd.concat([numeric_data22, textual_data22], axis=1)
cleaned_players_22

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,30.0,...,119,119,119,117,99,56,56,56,99,18
1,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,9.0,...,131,131,131,107,99,105,105,105,99,18
2,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,7.0,...,88,88,88,102,93,68,68,68,93,20
3,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,10.0,...,113,113,113,123,104,56,56,56,104,20
4,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,17.0,...,217,217,217,181,168,164,164,164,168,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,70000.0,1000.0,22,180,64,112541.0,1.0,36.0,...,47,47,47,45,49,46,46,46,49,9
19235,47,59,110000.0,500.0,19,175,70,445.0,1.0,27.0,...,47,47,47,48,49,42,42,42,49,7
19236,47,55,100000.0,500.0,21,178,72,111131.0,1.0,31.0,...,47,47,47,44,47,44,44,44,47,3
19237,47,60,110000.0,500.0,19,173,66,111131.0,1.0,12.0,...,15,15,15,25,25,11,11,11,25,9


In [87]:
features = cleaned_players_22[selected_features]
features

,potential,value_eur,age,release_clause_eur,dribbling,shooting,wage_eur
0,93,78000000.0,34,144300000.0,95.0,92.0,320000.0
1,92,119500000.0,32,197200000.0,86.0,92.0,270000.0
2,91,45000000.0,36,83300000.0,88.0,94.0,270000.0
3,91,129000000.0,29,238700000.0,94.0,83.0,270000.0
4,91,125500000.0,30,232200000.0,88.0,86.0,350000.0
...,...,...,...,...,...,...,...
19234,52,70000.0,22,114000.0,48.0,35.0,1000.0
19235,59,110000.0,19,193000.0,46.0,39.0,500.0
19236,55,100000.0,21,175000.0,49.0,37.0,500.0
19237,60,110000.0,19,239000.0,48.0,46.0,500.0


In [88]:
features_scaled = scaler.fit_transform(features)

In [99]:
new_data_predgb = pd.DataFrame(final_gb_model.predict(features_scaled))
new_data_predgb

,0
0,90.548494
1,91.638004
2,89.031996
3,90.352880
4,90.917621
...,...
19234,52.241583
19235,53.191451
19236,54.011106
19237,53.799597


In [104]:
new_data_predxgb = pd.DataFrame(final_xgb_model.predict(features_scaled))
new_data_predxgb

,0
0,90.435997
1,91.689842
2,89.919113
3,90.148674
4,90.559601
...,...
19234,51.684105
19235,51.689598
19236,52.190372
19237,51.994064


In [91]:
new_data_pred = pd.DataFrame(final_rfr_model.predict(features_scaled))
new_data_pred

,0
0,89.940000
1,90.420000
2,89.125000
3,90.385000
4,90.695000
...,...
19234,52.965000
19235,53.130000
19236,52.910000
19237,53.155988


In [92]:
cleaned_players_22['overall']

0        93
1        92
2        91
3        91
4        91
         ..
19234    47
19235    47
19236    47
19237    47
19238    47
Name: overall, Length: 19239, dtype: int64

In [93]:
#evaluating the model's performance on the new data
ground_truth = cleaned_players_22['overall']
mean_squared_error(ground_truth, new_data_pred)

1.0852271198090129

In [113]:
prediction_rootmse = np.sqrt(mean_squared_error(ground_truth, new_data_predxgb))
prediction_rootmse


1.08775861963159

SAVING THE TRAINED AND TESTED MODEL:

In [95]:
model_directory = '/content/drive/MyDrive/Colab Notebooks/final_xgb_model.pk1'

with open(model_directory, 'wb') as file:
  pickle.dump(final_xgb_model, file)

In [96]:
joblib.dump(scaler, '/content/drive/MyDrive/Colab Notebooks/scaler.pkl')


['/content/drive/MyDrive/Colab Notebooks/scaler.pkl']